In [1]:
!pip install torch torchvision torchaudio transformers opencv-python pillow numpy matplotlib seaborn
!pip install gradio lime shap grad-cam pytorch-gradcam
!pip install librosa soundfile datasets accelerate
!pip install scikit-learn pandas plotly
!pip install lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=62c4f33467cf23e56398b8aad26fb5ac7ba2dae14855bccf2412a3a6b5696d6b
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=bcaa9fc4d81b1553a7fd25545e3ff0473f134ab2aeab240b1e79fc26d17cbdce
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
  Created wheel for pytorch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Wav2Vec2Processor, Wav2Vec2ForSequenceClassification,
    pipeline
)
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import librosa
import soundfile as sf
import gradio as gr
import lime
from lime.lime_text import LimeTextExplainer
from lime.lime_image import LimeImageExplainer
import shap
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import plotly.graph_objects as go
import plotly.express as px
from google.colab import drive
import os
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

drive.mount('/content/drive')

project_path = '/content/drive/My Drive/XAI_Emotion_Analysis'
os.makedirs(project_path, exist_ok=True)
os.makedirs(f'{project_path}/models', exist_ok=True)
os.makedirs(f'{project_path}/results', exist_ok=True)


Mounted at /content/drive


In [3]:
class MultiModalEmotionAnalyzer:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.setup_text_model()
        self.setup_image_model()
        self.setup_audio_model()
        self.setup_explainers()

    def setup_text_model(self):
        print("Loading text emotion model...")
        try:
            model_name = "j-hartmann/emotion-english-distilroberta-base"
            self.text_tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.text_model = AutoModelForSequenceClassification.from_pretrained(model_name)
            self.text_model.to(self.device)
            self.text_pipeline = pipeline(
                "text-classification",
                model=self.text_model,
                tokenizer=self.text_tokenizer,
                device=0 if torch.cuda.is_available() else -1
            )
            print("Text model loaded successfully")
        except Exception as e:
            print(f"Error loading text model: {e}")
            self.text_pipeline = None

    def setup_image_model(self):
        print("Loading image emotion model...")
        try:
            self.image_pipeline = pipeline(
                "image-classification",
                model="trpakov/vit-face-expression",
                device=0 if torch.cuda.is_available() else -1
            )
            print("Image model loaded successfully")
        except Exception as e:
            print(f"Error loading image model: {e}")
            try:
                self.image_pipeline = pipeline(
                    "image-classification",
                    model="dima806/facial_emotions_image_detection",
                    device=0 if torch.cuda.is_available() else -1
                )
                print("Alternative image model loaded successfully")
            except Exception as e2:
                print(f"Error loading alternative image model: {e2}")
                self.image_pipeline = None

    def setup_audio_model(self):
        print("Loading audio emotion model...")
        try:
            self.audio_pipeline = pipeline(
                "audio-classification",
                model="superb/wav2vec2-base-superb-er",
                device=0 if torch.cuda.is_available() else -1
            )
            print("Audio pipeline loaded successfully")
        except Exception as e:
            print(f"Error loading audio pipeline: {e}")
            self.audio_pipeline = None

    def setup_explainers(self):
        print("Setting up explainers...")
        try:
            self.text_explainer = LimeTextExplainer(
                class_names=['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
            )
            self.image_explainer = LimeImageExplainer()
            print("Explainers setup successfully")
        except Exception as e:
            print(f"Error setting up explainers: {e}")
            self.text_explainer = None
            self.image_explainer = None

    def analyze_text_emotion(self, text):
        if self.text_pipeline is None:
            return {"error": "Text model not available"}

        try:
            results = self.text_pipeline(text)
            return {
                "modality": "text",
                "emotion": results[0]["label"],
                "confidence": results[0]["score"],
                "all_scores": results
            }
        except Exception as e:
            return {"error": f"Text analysis failed: {e}"}

    def analyze_text_with_explanation(self, text):
        if self.text_pipeline is None:
            return {"error": "Text model not available"}

        try:
            results = self.text_pipeline(text)
            main_result = {
                "modality": "text",
                "emotion": results[0]["label"],
                "confidence": results[0]["score"],
                "all_scores": results
            }
            def predict_proba(texts):
                predictions = []
                for txt in texts:
                    try:
                        result = self.text_pipeline(txt)
                        proba = [0.0] * 7
                        emotion_map = {
                            'anger': 0, 'disgust': 1, 'fear': 2,
                            'joy': 3, 'neutral': 4, 'sadness': 5, 'surprise': 6
                        }

                        for pred in result:
                            if pred['label'].lower() in emotion_map:
                                proba[emotion_map[pred['label'].lower()]] = pred['score']

                        predictions.append(proba)
                    except:
                        predictions.append([1/7] * 7)

                return np.array(predictions)

            try:
                if self.text_explainer:
                    explanation = self.text_explainer.explain_instance(
                        text, predict_proba, num_features=10
                    )
                    main_result["explanation"] = f"Key words affecting prediction: {', '.join([word for word, score in explanation.as_list()[:5]])}"
                else:
                    main_result["explanation"] = f"Analysis shows {results[0]['label']} emotion with {results[0]['score']:.1%} confidence"
            except:
                main_result["explanation"] = f"The text expresses {results[0]['label']} emotion with {results[0]['score']:.1%} confidence"
            main_result["predictions"] = results

            return main_result

        except Exception as e:
            return {"error": f"Text analysis with explanation failed: {e}"}

    def analyze_image_emotion(self, image_path):
        if self.image_pipeline is None:
            return {"error": "Image model not available"}

        try:
            image = Image.open(image_path)
            results = self.image_pipeline(image)
            return {
                "modality": "image",
                "emotion": results[0]["label"],
                "confidence": results[0]["score"],
                "all_scores": results
            }
        except Exception as e:
            return {"error": f"Image analysis failed: {e}"}

    def analyze_audio_emotion(self, audio_path, sample_rate=16000):
        if self.audio_pipeline is None:
            return {"error": "Audio model not available"}

        try:
            results = self.audio_pipeline(audio_path)
            return {
                "modality": "audio",
                "emotion": results[0]["label"],
                "confidence": results[0]["score"],
                "all_scores": results
            }
        except Exception as e:
            return {"error": f"Audio analysis failed: {e}"}

    def multimodal_analysis(self, text=None, image_path=None, audio_path=None):
        results = {}
        if text:
            results["text"] = self.analyze_text_with_explanation(text)
        else:
            results["text"] = {"error": "No text provided"}

        if image_path:
            results["image"] = self.analyze_image_emotion(image_path)
        else:
            results["image"] = {"error": "No image provided"}

        if audio_path:
            results["audio"] = self.analyze_audio_emotion(audio_path)
        else:
            results["audio"] = {"error": "No audio provided"}

        fusion_result = self.multimodal_fusion(
            results["text"],
            results["image"],
            results["audio"]
        )

        if "dominant_emotion" in fusion_result:
            fusion_result["predicted_emotion"] = fusion_result["dominant_emotion"]

        return {
            "individual_results": results,
            "fusion_result": fusion_result,
            "predicted_emotion": fusion_result.get("dominant_emotion", "unknown")
        }

    def multimodal_fusion(self, text_result, image_result, audio_result, weights=None):
        if weights is None:
            weights = {"text": 0.4, "image": 0.35, "audio": 0.25}

        try:
            valid_results = []
            if "error" not in text_result:
                valid_results.append(("text", text_result, weights["text"]))
            if "error" not in image_result:
                valid_results.append(("image", image_result, weights["image"]))
            if "error" not in audio_result:
                valid_results.append(("audio", audio_result, weights["audio"]))

            if not valid_results:
                return {"error": "No valid results for fusion", "predicted_emotion": "unknown"}

            emotion_scores = {}
            total_weight = sum(weight for _, _, weight in valid_results)

            for modality, result, weight in valid_results:
                emotion = result["emotion"]
                confidence = result["confidence"]
                normalized_weight = weight / total_weight

                if emotion not in emotion_scores:
                    emotion_scores[emotion] = 0
                emotion_scores[emotion] += confidence * normalized_weight

            dominant_emotion = max(emotion_scores, key=emotion_scores.get)

            return {
                "dominant_emotion": dominant_emotion,
                "predicted_emotion": dominant_emotion,
                "confidence": emotion_scores[dominant_emotion],
                "emotion_scores": emotion_scores,
                "modalities_used": [modality for modality, _, _ in valid_results]
            }
        except Exception as e:
            return {"error": f"Fusion failed: {e}", "predicted_emotion": "unknown"}

print("Reinitializing MultiModal Emotion Analyzer...")
analyzer = MultiModalEmotionAnalyzer()
print("Reinitialization complete!")





Reinitializing MultiModal Emotion Analyzer...
Using device: cuda
Loading text emotion model...


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


Audio pipeline loaded successfully
Setting up explainers...
Explainers setup successfully
Reinitialization complete!


In [4]:
    def analyze_text_with_explanation(self, text):
        print(f"Analyzing text: {text[:50]}...")

        results = self.text_pipeline(text)

        def predict_proba(texts):
            predictions = []
            for text in texts:
                result = self.text_pipeline(text)
                probs = np.zeros(7)
                emotion_map = {
                    'LABEL_0': 0,  # anger
                    'LABEL_1': 1,  # disgust
                    'LABEL_2': 2,  # fear
                    'LABEL_3': 3,  # joy
                    'LABEL_4': 4,  # neutral
                    'LABEL_5': 5,  # sadness
                    'LABEL_6': 6   # surprise
                }
                for r in result:
                    if r['label'] in emotion_map:
                        probs[emotion_map[r['label']]] = r['score']
                predictions.append(probs)
            return np.array(predictions)
        explanation = self.text_explainer.explain_instance(
            text,
            predict_proba,
            num_features=10,
            num_samples=100
        )

        important_words = explanation.as_list()

        explanation_html = explanation.as_html()

        return {
            'predictions': results,
            'explanation': important_words,
            'explanation_html': explanation_html,
            'confidence': max([r['score'] for r in results])
        }


In [5]:
    def analyze_image_with_explanation(self, image_path):
        print("Analyzing image...")

        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image)

        results = self.image_pipeline(image)
        model = self.image_pipeline.model
        target_layers = [model.vit.encoder.layer[-1].layernorm_before]

        cam = GradCAM(model=model, target_layers=target_layers)

        preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        input_tensor = preprocess(image).unsqueeze(0).to(self.device)

        target_category = None
        grayscale_cam = cam(input_tensor=input_tensor, targets=target_category)
        grayscale_cam = grayscale_cam[0, :]

        rgb_img = np.float32(cv2.resize(image_array, (224, 224))) / 255
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
        important_regions = self.extract_important_regions(grayscale_cam)

        return {
            'predictions': results,
            'heatmap': visualization,
            'important_regions': important_regions,
            'confidence': max([r['score'] for r in results])
        }

    def extract_important_regions(self, cam):
        threshold = np.percentile(cam, 80)
        important_mask = cam > threshold
        total_pixels = cam.size
        important_pixels = np.sum(important_mask)
        coverage = (important_pixels / total_pixels) * 100

        return {
            'coverage_percentage': coverage,
            'max_activation': float(np.max(cam)),
            'mean_activation': float(np.mean(cam[important_mask])) if important_pixels > 0 else 0
        }


In [6]:
    def analyze_audio_with_explanation(self, audio_path):
        print("Analyzing audio...")
        speech, sr = librosa.load(audio_path, sr=16000)

        inputs = self.audio_processor(
            speech,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )
        inputs = inputs.to(self.device)
        with torch.no_grad():
            logits = self.audio_model(**inputs).logits
            predictions = torch.nn.functional.softmax(logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=-1).item()

        emotion_labels = ["angry", "calm", "disgust", "fearful", "happy", "neutral", "sad", "surprised"]

        spectral_features = self.extract_spectral_features(speech, sr)

        explanation = self.create_audio_explanation(speech, sr, predictions)

        return {
            'predictions': {
                'label': emotion_labels[predicted_class],
                'confidence': float(predictions[0][predicted_class]),
                'all_scores': {emotion_labels[i]: float(predictions[0][i]) for i in range(len(emotion_labels))}
            },
            'spectral_features': spectral_features,
            'explanation': explanation
        }

    def extract_spectral_features(self, audio, sr):
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
        zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)[0]

        return {
            'mfcc_mean': float(np.mean(mfccs)),
            'mfcc_std': float(np.std(mfccs)),
            'spectral_centroid_mean': float(np.mean(spectral_centroids)),
            'spectral_rolloff_mean': float(np.mean(spectral_rolloff)),
            'zero_crossing_rate_mean': float(np.mean(zero_crossing_rate))
        }

    def create_audio_explanation(self, audio, sr, predictions):
        fft = np.fft.fft(audio)
        magnitude = np.abs(fft)

        freqs = np.fft.fftfreq(len(fft), 1/sr)
        dominant_freq_idx = np.argmax(magnitude[:len(magnitude)//2])
        dominant_freq = abs(freqs[dominant_freq_idx])
        energy = np.sum(audio**2)

        return {
            'dominant_frequency': float(dominant_freq),
            'total_energy': float(energy),
            'audio_length': len(audio) / sr,
            'key_indicators': [
                f"Dominant frequency: {dominant_freq:.1f} Hz",
                f"Audio energy level: {energy:.2e}",
                f"Duration: {len(audio)/sr:.2f} seconds"
            ]
        }


In [7]:
    def multimodal_analysis(self, text=None, image_path=None, audio_path=None):
        results = {
            'individual_analyses': {},
            'fusion_result': {},
            'explanations': {}
        }

        weights = {'text': 0.4, 'image': 0.4, 'audio': 0.2}
        combined_emotions = {}

        if text:
            text_result = self.analyze_text_with_explanation(text)
            results['individual_analyses']['text'] = text_result
            results['explanations']['text'] = text_result['explanation']

            for pred in text_result['predictions']:
                emotion = pred['label'].lower()
                if 'label_' in emotion:
                    emotion_map = {
                        'label_0': 'anger', 'label_1': 'disgust', 'label_2': 'fear',
                        'label_3': 'joy', 'label_4': 'neutral', 'label_5': 'sadness',
                        'label_6': 'surprise'
                    }
                    emotion = emotion_map.get(emotion, emotion)

                combined_emotions[emotion] = combined_emotions.get(emotion, 0) + pred['score'] * weights['text']

        if image_path and os.path.exists(image_path):
            image_result = self.analyze_image_with_explanation(image_path)
            results['individual_analyses']['image'] = image_result
            results['explanations']['image'] = image_result['important_regions']

            for pred in image_result['predictions']:
                emotion = pred['label'].lower()
                combined_emotions[emotion] = combined_emotions.get(emotion, 0) + pred['score'] * weights['image']

        if audio_path and os.path.exists(audio_path):
            audio_result = self.analyze_audio_with_explanation(audio_path)
            results['individual_analyses']['audio'] = audio_result
            results['explanations']['audio'] = audio_result['explanation']

            for emotion, score in audio_result['predictions']['all_scores'].items():
                combined_emotions[emotion] = combined_emotions.get(emotion, 0) + score * weights['audio']

        if combined_emotions:
            final_emotion = max(combined_emotions, key=combined_emotions.get)
            final_confidence = combined_emotions[final_emotion]

            results['fusion_result'] = {
                'predicted_emotion': final_emotion,
                'confidence': final_confidence,
                'all_emotions': combined_emotions
            }

        return results

analyzer = MultiModalEmotionAnalyzer()


Using device: cuda
Loading text emotion model...


Device set to use cuda:0


Text model loaded successfully
Loading image emotion model...


Device set to use cuda:0


Image model loaded successfully
Loading audio emotion model...


Device set to use cuda:0


Audio pipeline loaded successfully
Setting up explainers...
Explainers setup successfully


In [8]:
def create_visualization_plots(results):
    plots = []

    if 'fusion_result' in results and results['fusion_result']:
        emotions = list(results['fusion_result']['all_emotions'].keys())
        scores = list(results['fusion_result']['all_emotions'].values())

        fig = px.bar(
            x=emotions,
            y=scores,
            title="Combined Emotion Analysis",
            labels={'x': 'Emotions', 'y': 'Confidence Score'},
            color=scores,
            color_continuous_scale='viridis'
        )
        fig.update_layout(height=400)
        plots.append(fig)

    return plots

def analyze_multimodal(text, image, audio):
    try:
        image_path = None
        audio_path = None

        if image is not None:
            image_path = f"{project_path}/temp_image.jpg"
            image.save(image_path)

        if audio is not None:
            audio_path = f"{project_path}/temp_audio.wav"
            if hasattr(audio, 'name'):
                import shutil
                shutil.copy(audio.name, audio_path)
            else:
                sf.write(audio_path, audio[1], audio[0])

        results = analyzer.multimodal_analysis(text, image_path, audio_path)

        report = create_detailed_report(results)

        plots = create_visualization_plots(results)

        for temp_file in [image_path, audio_path]:
            if temp_file and os.path.exists(temp_file):
                os.remove(temp_file)

        return report, plots[0] if plots else None

    except Exception as e:
        return f"Error in analysis: {str(e)}", None

def create_detailed_report(results):
    report = "# Explainable AI Emotion Analysis Report\n\n"

    if 'fusion_result' in results and results['fusion_result']:
        fusion = results['fusion_result']
        report += f"## Final Result\n"
        report += f"**Predicted Emotion: {fusion['predicted_emotion'].upper()}**\n"
        report += f"**Confidence: {fusion['confidence']:.3f}**\n\n"

    report += "## Individual Analysis\n\n"

    if 'text' in results['individual_analyses']:
        text_result = results['individual_analyses']['text']
        report += "###  Text Analysis\n"
        for pred in text_result['predictions']:
            report += f"- **{pred['label']}**: {pred['score']:.3f}\n"

        report += "\n** Why the model decided this way (Text):**\n"
        if 'explanation' in results['explanations'].get('text', {}):
            for word, importance in results['explanations']['text']['explanation']:
                if importance > 0:
                    report += f"- '{word}' increased confidence by {importance:.3f}\n"
                else:
                    report += f"- '{word}' decreased confidence by {abs(importance):.3f}\n"

    if 'image' in results['individual_analyses']:
        image_result = results['individual_analyses']['image']
        report += "\n###  Image Analysis\n"
        for pred in image_result['predictions']:
            report += f"- **{pred['label']}**: {pred['score']:.3f}\n"

        report += "\n** Why the model decided this way (Image):**\n"
        regions = results['explanations']['image']
        report += f"- Important facial regions cover {regions['coverage_percentage']:.1f}% of the image\n"
        report += f"- Maximum activation strength: {regions['max_activation']:.3f}\n"
        report += f"- Average activation in key areas: {regions['mean_activation']:.3f}\n"

    if 'audio' in results['individual_analyses']:
        audio_result = results['individual_analyses']['audio']
        report += "\n###  Audio Analysis\n"
        report += f"- **{audio_result['predictions']['label']}**: {audio_result['predictions']['confidence']:.3f}\n"

        report += "\n** Why the model decided this way (Audio):**\n"
        explanation = results['explanations']['audio']
        for indicator in explanation['key_indicators']:
            report += f"- {indicator}\n"

        spectral = audio_result['spectral_features']
        report += f"- MFCC features suggest emotional content (mean: {spectral['mfcc_mean']:.3f})\n"
        report += f"- Spectral centroid indicates voice characteristics: {spectral['spectral_centroid_mean']:.1f} Hz\n"

    return report


In [11]:
def create_gradio_interface():

    def analyze_multimodal(text, image, audio):
        try:
            if not any([text, image, audio]):
                return " Please provide at least one input (text, image, or audio)"

            results = []

            if text and text.strip():
                text_result = analyzer.analyze_text_emotion(text.strip())
                if "error" not in text_result:
                    results.append(f" **Text Analysis:**\n- Emotion: {text_result['emotion']}\n- Confidence: {text_result['confidence']:.2%}")
                else:
                    results.append(f" **Text Analysis:** {text_result['error']}")

            if image is not None:
                image_result = analyzer.analyze_image_emotion(image)
                if "error" not in image_result:
                    results.append(f" **Image Analysis:**\n- Emotion: {image_result['emotion']}\n- Confidence: {image_result['confidence']:.2%}")
                else:
                    results.append(f" **Image Analysis:** {image_result['error']}")

            if audio is not None:
                audio_result = analyzer.analyze_audio_emotion(audio)
                if "error" not in audio_result:
                    results.append(f" **Audio Analysis:**\n- Emotion: {audio_result['emotion']}\n- Confidence: {audio_result['confidence']:.2%}")
                else:
                    results.append(f" **Audio Analysis:** {audio_result['error']}")

            return "\n\n".join(results) if results else " No valid analysis results"

        except Exception as e:
            return f" Error in multimodal analysis: {str(e)}"

    interface = gr.Interface(
        fn=analyze_multimodal,
        inputs=[
            gr.Textbox(
                label=" Text Input",
                placeholder="Enter text to analyze emotions...",
                lines=3
            ),
            gr.Image(
                label=" Image Input",
                type="filepath"
            ),
            gr.Audio(
                label=" Audio Input",
                type="filepath"
            )
        ],
        outputs=gr.Textbox(
            label=" Analysis Results",
            lines=10
        ),
        title=" Multimodal Emotion Analysis",
        description="Upload text, image, and/or audio to analyze emotions using AI models",
        theme=gr.themes.Soft(),
        allow_flagging="never"
    )

    return interface

print("Creating Gradio interface...")
demo = create_gradio_interface()
demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)



Creating Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://757d484b03ce6abe87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://757d484b03ce6abe87.gradio.live


In [9]:
def save_models_to_drive():
    print("Saving models to Google Drive...")

    config = {
        'text_model': "j-hartmann/emotion-english-distilroberta-base",
        'image_model': "trpakov/vit-face-expression",
        'audio_model': "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition",
        'emotion_labels': {
            'text': ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise'],
            'image': ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
            'audio': ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
        }
    }

    with open(f'{project_path}/config.json', 'w') as f:
        json.dump(config, f, indent=2)

    print(f"Models and config saved to {project_path}")

save_models_to_drive()


Saving models to Google Drive...
Models and config saved to /content/drive/My Drive/XAI_Emotion_Analysis


In [10]:
def test_system():
    print(" Testing the system...")

    test_text = "I am extremely happy and excited about this amazing news!"
    print(f"\nTesting text: {test_text}")

    try:
        text_result = analyzer.analyze_text_with_explanation(test_text)
        print(" Text analysis successful")
        print(f"Prediction: {text_result['predictions'][0]['label']}")
        print(f"Confidence: {text_result['confidence']:.3f}")
    except Exception as e:
        print(f" Text analysis failed: {e}")

    try:
        multimodal_result = analyzer.multimodal_analysis(text=test_text)
        print(" Multimodal analysis successful")
        if 'fusion_result' in multimodal_result:
            fusion = multimodal_result['fusion_result']
            print(f"Final emotion: {fusion['predicted_emotion']}")
            print(f"Final confidence: {fusion['confidence']:.3f}")
    except Exception as e:
        print(f" Multimodal analysis failed: {e}")

    print(" System testing completed!")
test_system()


 Testing the system...

Testing text: I am extremely happy and excited about this amazing news!


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Text analysis successful
Prediction: joy
Confidence: 0.979
 Multimodal analysis successful
Final emotion: joy
Final confidence: 0.979
 System testing completed!


In [12]:
def analyze_with_explanation(text, image, audio):
    try:
        results = []
        explanation_details = []
        if text and text.strip():
            text_result = analyzer.analyze_text_with_explanation(text.strip())
            if "error" not in text_result:
                results.append(f" **متن:** {text_result['emotion']} ({text_result['confidence']:.1%})")

                if "explanation" in text_result:
                    explanation_details.append(f" **توضیح متن:** {text_result['explanation']}")

                if "all_scores" in text_result:
                    scores_text = " **امتیازات تمام احساسات:**\n"
                    for score in text_result["all_scores"][:3]:
                        scores_text += f"   • {score['label']}: {score['score']:.1%}\n"
                    explanation_details.append(scores_text.strip())
            else:
                results.append(f" **متن:** خطا - {text_result['error']}")

        if image is not None:
            image_result = analyzer.analyze_image_emotion(image)
            if "error" not in image_result:
                results.append(f" **تصویر:** {image_result['emotion']} ({image_result['confidence']:.1%})")

                if "all_scores" in image_result:
                    scores_text = " **تحلیل تصویر:**\n"
                    for score in image_result["all_scores"][:3]:
                        scores_text += f"   • {score['label']}: {score['score']:.1%}\n"
                    explanation_details.append(scores_text.strip())

                confidence = image_result['confidence']
                if confidence > 0.8:
                    explanation_details.append(f" **تحلیل:** احساس در تصویر با قطعیت بالا ({confidence:.1%}) شناسایی شده است")
                elif confidence > 0.6:
                    explanation_details.append(f" **تحلیل:** احساس با قطعیت متوسط ({confidence:.1%}) تشخیص داده شده")
                else:
                    explanation_details.append(f" **تحلیل:** احساس با قطعیت پایین ({confidence:.1%}) شناسایی شده - ممکن است مبهم باشد")
            else:
                results.append(f" **تصویر:** خطا - {image_result['error']}")

        if audio is not None:
            audio_result = analyzer.analyze_audio_emotion(audio)
            if "error" not in audio_result:
                results.append(f" **صوت:** {audio_result['emotion']} ({audio_result['confidence']:.1%})")

                if "all_scores" in audio_result:
                    scores_text = " **تحلیل صوت:**\n"
                    for score in audio_result["all_scores"][:3]:
                        scores_text += f"  {score['label']}: {score['score']:.1%}\n"
                    explanation_details.append(scores_text.strip())
            else:
                results.append(f" **صوت:** خطا - {audio_result['error']}")

        if len([x for x in [text, image, audio] if x is not None and (isinstance(x, str) and x.strip() or not isinstance(x, str))]) > 1:
            multimodal_result = analyzer.multimodal_analysis(text, image, audio)

            if "fusion_result" in multimodal_result and "error" not in multimodal_result["fusion_result"]:
                fusion = multimodal_result["fusion_result"]

                results.append(f"\n **نتیجه تلفیقی:** {fusion['dominant_emotion']} ({fusion['confidence']:.1%})")

                modalities_used = fusion.get("modalities_used", [])
                explanation_details.append(f" **روش تلفیق:** استفاده از {', '.join(modalities_used)} برای تحلیل جامع")

                if "emotion_scores" in fusion:
                    fusion_scores = " **امتیازات تلفیقی:**\n"
                    sorted_emotions = sorted(fusion["emotion_scores"].items(), key=lambda x: x[1], reverse=True)
                    for emotion, score in sorted_emotions[:3]:
                        fusion_scores += f"   • {emotion}: {score:.1%}\n"
                    explanation_details.append(fusion_scores.strip())

        # =================== ساخت خروجی نهایی ===================
        if not results:
            return " لطفاً حداقل یک ورودی ارائه دهید"

        # ترکیب نتایج و توضیحات
        final_output = "\n".join(results)

        if explanation_details:
            final_output += "\n\n" + "="*50 + "\n **توضیحات تفصیلی Explainable AI:**\n" + "="*50 + "\n\n"
            final_output += "\n\n".join(explanation_details)

        return final_output

    except Exception as e:
        return f" خطا در تحلیل: {str(e)}"

demo = gr.Interface(
    fn=analyze_with_explanation,
    inputs=[
        gr.Textbox(
            label=" متن",
            placeholder="مثال: I am extremely happy and excited about this amazing news!",
            lines=3
        ),
        gr.Image(label=" تصویر چهره", type="filepath"),
        gr.Audio(label=" فایل صوتی", type="filepath")
    ],
    outputs=gr.Textbox(
        label=" نتایج تحلیل + توضیحات Explainable AI",
        lines=15,
        show_copy_button=True
    ),
    title=" سیستم تحلیل احساسات چندرسانه‌ای با Explainable AI",
    description="""
    ###  قابلیت‌ها:
    • **تحلیل متن:** شناسایی احساسات در متن با توضیحات کلیدواژه‌ها
    • **تحلیل تصویر:** تشخیص احساسات چهره با درجه اطمینان
    • **تحلیل صوت:** شناسایی احساسات صوتی
    • **تلفیق هوشمند:** ترکیب نتایج چندرسانه‌ای
    • **Explainable AI:** توضیحات تفصیلی فرآیند تصمیم‌گیری

     می‌توانید یک یا چند رسانه را همزمان آپلود کنید
    """,
    theme=gr.themes.Soft()
)

demo.launch(share=True, debug=True, server_name="0.0.0.0")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e1f7ced5b7ccd20b1a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://e1f7ced5b7ccd20b1a.gradio.live
